<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/9_Create_DNN_District_Test_Generic_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set-up
import pandas as pd
import numpy as np
from google.colab import files

pd.set_option('display.max_columns', None)


def one_hot(df, column):
    o_h = pd.get_dummies(df[column])
    new = pd.concat([df, o_h], axis=1)
    new = new.drop([column], axis=1)
    return new
    

def dlcsv(df,name):
  df.to_csv('{}.csv'.format(name), index=False)
  files.download('{}.csv'.format(name))

In [ ]:
#combine weather and crime for each year
data= []

for year in range(2015,2020):
  weather = pd.read_csv('https://raw.githubusercontent.com/MattIzon/16010269_DataAnalytics/main/data/w_{}.csv'.format(year)).set_index('date')
  crime = pd.read_csv('https://raw.githubusercontent.com/MattIzon/16010269_DataAnalytics/main/data/c_district_{}.csv'.format(year)).set_index('date')
  crime.drop('Unnamed: 0', axis=1, inplace=True)
  data.append(crime.join(weather).reset_index())

In [ ]:
#add year column
year = 2015
for df in data:
  df['year'] = year
  year += 1

In [ ]:
# Combine all years into single dataset
combined = pd.concat(data).reset_index()
combined.drop('index', axis=1, inplace=True)

In [ ]:
# one hot encode day_of_week
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

for index in range(len(days)):
  value = index + 1
  combined.loc[combined.day_of_week == value, 'day_of_week'] = days[index]

combined = one_hot(combined, 'day_of_week')

In [ ]:
# one hot encode mo
days = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for index in range(len(days)):
  value = index + 1
  combined.loc[combined.mo == value, 'mo'] = days[index]

combined = one_hot(combined, 'mo')

In [ ]:
# one hot encode district
combined.district = df.district.apply(np.int64)
districts = list(combined.district.unique())
combined = one_hot(combined, 'district')
for district in districts:
  combined.rename({district: 'district_{}'.format(district)}, axis=1, inplace=True)

In [ ]:
# Shuffle the dataset
rand_combined = combined.iloc[np.random.permutation(len(combined))]

In [ ]:
# Save testing and generic datasets
test_size = int(len(rand_combined)*0.01)
test_set = rand_combined.iloc[:test_size]
dlcsv(test_set, 'DNN_test_set')

generic_set = rand_combined.iloc[test_size:]
dlcsv(generic_set, 'DNN_generic_set')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>